# Load Libraries 

In [1]:
%pip install segmentation-models &> /dev/null
%load_ext tensorboard
!cp '/content/drive/MyDrive/Transfere Learning research/Notebooks/unet_util.py' .

import shutil
import numpy as np
import tensorflow as tf
from keras import backend as K
import segmentation_models as sm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from unet_util import dice_coef_loss, dice_coef, jacard_coef, dice_coef_loss, Residual_CNN_block, multiplication, attention_up_and_concatenate, multiplication2, attention_up_and_concatenate2, UNET_224, evaluate_prediction_result

sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

### Define nesscesary paths 

- Data folder
- Path to save models
- Path to save prediction data

In [2]:
% cd '/content/drive/MyDrive/Transfere Learning research/Notebooks/'
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)

# Avaiable backbones for Unet architechture
# 'vgg16' 'vgg19' 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152' 'inceptionv3' 
# 'inceptionresnetv2' 'densenet121' 'densenet169' 'densenet201' 'seresnet18' 'seresnet34' 
# 'seresnet50' 'seresnet101' 'seresnet152', and 'attentionUnet'
backend = 'resnet101'

# Data location
# 'covington' 'rowancreek' 'alaska'
location = 'covington'

# Fine-tuning flag
# True/False
finetune = True

# Construct the model save file name
name = location+'-Unet-'+backend + ('-tf' if(finetune) else '')

logdir = log_path + name
if(os.path.isdir(logdir)):
  shutil.rmtree(logdir)
os.makedirs(logdir, exist_ok=True)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

print('model location: '+ model_path+name+'.h5')

/content/drive/MyDrive/Transfere Learning research/Notebooks
model location: ./models/covington-Unet-resnet101-tf.h5


# Create the model with selected backend

In [3]:
# Create U-net model with the chosen backbone
# The U-net will be initialized with ImageNet weights and the ImageNet weights will be frozen 
# in the first pass training

if (backend=="attentionUnet"):
  # Attention U-net model
  learning_rate =0.0000359
  model = UNET_224()
  model.compile(optimizer = Adam(learning_rate=learning_rate), 
                loss = dice_coef_loss, 
                metrics = [dice_coef,'accuracy'])
else:
  # Unet with ImageNet backends
  base_model = sm.Unet(backend, classes = 1, encoder_weights = 'imagenet', encoder_freeze = finetune)
  
  # The backbones are trained RGB so we need to add new input wiht 8 channels
  # Conv2D will convert 8 channels input to 3 channels input for the pretrained backbones
  inp = Input(shape=(None, None, 8))
  l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
  out = base_model(l1)
  model = Model(inp, out, name = base_model.name)
  
  # Compile the model with 'Adam' optimizer (0.001 is the default learning rate) and define the loss and metrics
  model.compile(optimizer = Adam(), loss = dice_coef_loss, metrics=[dice_coef,'accuracy'])

# define hyperparameters and callback modules
patience = 10
maxepoch = 500
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
             EarlyStopping(monitor='val_loss', patience=patience, verbose=0),
             ModelCheckpoint(model_path+name+'.h5', monitor='val_loss', save_best_only=True, verbose=0),
             TensorBoard(log_dir=logdir)]

171180032/171164896 [==============================] - 6s 0us/step


# Training

- load data 
- model.fit 
  - save the model in ./models/



In [4]:
X_train = np.load(input_data+location+'/train_data.npy').astype(np.float32)
Y_train = np.load(input_data+location+'/train_label.npy').astype(np.float32)
X_validation = np.load(input_data+location+'/vali_data.npy').astype(np.float32)
Y_validation = np.load(input_data+location+'/vali_label.npy').astype(np.float32)

### First pass 

- learning rate: 0.001

In [5]:
train_history = model.fit(x = X_train,y = Y_train, 
                          validation_data = (X_validation, Y_validation), 
                          batch_size = 16, epochs = maxepoch, verbose=0, callbacks = callbacks)


Epoch 35: ReduceLROnPlateau reducing learning rate to 0.0007000000332482159.


### Second pass 
- learning rate 0.00001

In [6]:
if(finetune and backend != "attentionUnet"):

  # For fine-tuning we need to set the tranable flag to true for the whole model
  model.trainable = True

  # Recompile the model with the smaller learning rate at the optimizer (Adam(1e-5))
  model.compile(optimizer = Adam(1e-5), loss = dice_coef_loss, metrics=[dice_coef,'accuracy'])

  # train the model again
  train_history_2 = model.fit(x = X_train, y = Y_train,
                              validation_data=(X_validation, Y_validation),
                              batch_size=16,epochs=maxepoch,
                              initial_epoch = len(train_history.history['val_loss'])-1,
                              verbose=0 ,callbacks=callbacks)


Epoch 53: ReduceLROnPlateau reducing learning rate to 6.9999998231651255e-06.


# Evaluation

### predict the test data with the best model

In [7]:
# Load the best model saved by the callback module
from keras.models import load_model
if(backend != "attentionUnet"):
  best_model = load_model(model_path+name+'.h5',
                        custom_objects={'dice_coef_loss':dice_coef_loss,
                                        'dice_coef':dice_coef,})
else:
  best_model = load_model(model_path+name+'.h5',
                        custom_objects={'multiplication': multiplication,
                                        'multiplication2': multiplication2,
                                        'dice_coef_loss':dice_coef_loss, 
                                        'dice_coef':dice_coef,})

# load the test data
X_test = np.load(input_data+location+'/bottom_half_test_data.npy').astype(np.float32)

# predict the test data using the loaded model
test_predicted= best_model.predict(X_test)

# convert the prediction probability to true or false with threshold at 0.5
test_predicted_threshold = (test_predicted > 0.5).astype(np.uint8)

# save the prediction results
np.save(prediction_path+name+'_predict.npy',test_predicted_threshold)
print('Predtion results saved: ' + prediction_path+name+'_predict.npy')

Predtion results saved: ./predicts/covington-Unet-resnet101-tf_predict.npy


## Evaluate the prediction result

In [8]:
pred_npy = prediction_path+name+'_predict.npy'
mask_npy = input_data+location+'/bottom_half_test_mask.npy'
label_npy = input_data+location+'/bottom_half_test_label.npy'
model = model_path + name + '.h5'
text_path = prediction_path+'prediction_results.txt'

evaluate_prediction_result(location, pred_npy, mask_npy, label_npy, model, text_path)

Model path:./models/covington-Unet-resnet101-tf.h5  Run at: 2022-07-19 17:01:29.675759
F1 score of Nonstream: 0.9871383308037862
F1 score of Stream: 0.5177333889021195
Precision of Nonstream: 0.9954168762835196
Precision of Stream: 0.3946604640608158
Recall of Nonstream: 0.9789963492931095
Recall of Stream: 0.7523500529924513
